In [2]:
# Import webdriver from selenium
from selenium import webdriver
chrome_path = r'/Users/shanqin/Desktop/TextaAnalysis/chromedriver'
# prepend it with r because of the backslash characters

In [3]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd

In [4]:
def get_date(soup):
    date = []
    for element in soup.find_all(class_='MItem DateCreated'):
        date.append(element.find('time').text.strip())
    return date

In [5]:
def get_userid(soup):
    userid = []
    for element in soup.find_all(class_='Comment'):
        userid.append(element.find_all('span')[0].text.strip())
    return userid

In [14]:
def remove_quote(soup):
    cleantext = []
    text = soup.find_all('div',{'class':'Message'})
    for t in text:
        if t.find('blockquote'):
            try:
                cleantext.append(t.contents[2].strip())
            except:
                cleantext.append(u"")
        else:
            cleantext.append(t.contents[0].strip()) 
    return cleantext

In [28]:
def get_data (url, driver, pages):
    date = []
    userid = []
    cleantext = []
    for i in range (pages):
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        date += get_date(soup)
        userid += get_userid(soup)
        cleantext += remove_quote(soup)
        next_button = soup.find("a", class_="Previous Pager-nav")
        url = next_button['href']
    return pd.DataFrame(list(zip(date, userid, cleantext)))

In [30]:
url = "https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702"
driver = webdriver.Chrome(chrome_path)
get_data(url, driver, 5).to_csv("output.csv", index=False)